In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


Downloading and installing requirements for yolov7

In [ ]:
# # Download YOLOv7 repository and install requirements
# %cd /content/gdrive/MyDrive
# !git clone https://github.com/augmentedstartups/yolov7.git
# %cd yolov7
# !pip install -r requirements.txt

In [ ]:
%cd /content/gdrive/MyDrive/yolov7

/content/gdrive/MyDrive/yolov7


In [ ]:
!pwd

/content/gdrive/MyDrive/yolov7


Test the yolov7 installation

In [ ]:
#!python detect.py --weights yolov7.pt --source inference/images/horses.jpg --img 640

Train yolov7 with the custom data by giving necessary parameters.

In [ ]:
#!python3 train.py --weights yolov7.pt --data "/content/gdrive/MyDrive/smalldata/Stops.yaml" --workers 4 --batch-size 4 --img 416 --cfg cfg/training/yolov7.yaml --name yolov7 --hyp data/hyp.scratch.p5.yaml --epochs 100

Path to input Video

In [ ]:
input_path = "/content/gdrive/MyDrive/smalldata/VID3.mp4"

In [ ]:
# Retrieving input file name
import re
match = re.search(r"/([^/]+)\.mp4$", input_path)
if match:
    input_file = match.group(1)
    print(input_file)

VID3


In [ ]:
# delete previous experiment results
import os
import shutil
results_path = "/content/gdrive/MyDrive/yolov7/runs/detect/exp"
if os.path.exists(results_path):
    shutil.rmtree(results_path)
    print(f"Directory {results_path} deleted successfully.")

Directory /content/gdrive/MyDrive/yolov7/runs/detect/exp deleted successfully.


Detecting bounding boxes for Bus stop shelter and Bus stop sign board classes for input video by the trained model and store the coordinates of the detected classes.

In [ ]:
!python detect.py --weights runs/train/yolov7/weights/best.pt --source '{input_path}' --img 416 --save-txt

Namespace(weights=['runs/train/yolov7/weights/best.pt'], source='/content/gdrive/MyDrive/smalldata/VID3.mp4', img_size=416, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=True, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-104-g941b94c torch 2.0.0+cu118 CUDA:0 (Tesla T4, 15101.8125MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
Model Summary: 314 layers, 36487166 parameters, 6194944 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
# Delete folder to store the video frames
folder_path = '/content/gdrive/MyDrive/video_frames'
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Directory {folder_path} deleted successfully.")

Directory /content/gdrive/MyDrive/video_frames deleted successfully.


Divide the input video into frames and store it in a video_frames folder.

In [ ]:
# Open the video file
video = cv2.VideoCapture(input_path)

os.makedirs(folder_path)

# Loop through all the frames in the video
frame_num = 1
while True:
    # Read a frame from the video
    ret, frame = video.read()
    
    # If the frame cannot be read, break out of the loop
    if not ret:
        break

    filename = f'{folder_path}/{input_file}_{frame_num}.jpg'
    print(f"writing {filename}")
    cv2.imwrite(filename, frame)

    frame_num += 1

# Release the video object
video.release()

writing /content/gdrive/MyDrive/video_frames/VID3_1.jpg
writing /content/gdrive/MyDrive/video_frames/VID3_2.jpg
writing /content/gdrive/MyDrive/video_frames/VID3_3.jpg
writing /content/gdrive/MyDrive/video_frames/VID3_4.jpg
writing /content/gdrive/MyDrive/video_frames/VID3_5.jpg
writing /content/gdrive/MyDrive/video_frames/VID3_6.jpg
writing /content/gdrive/MyDrive/video_frames/VID3_7.jpg
writing /content/gdrive/MyDrive/video_frames/VID3_8.jpg
writing /content/gdrive/MyDrive/video_frames/VID3_9.jpg
writing /content/gdrive/MyDrive/video_frames/VID3_10.jpg
writing /content/gdrive/MyDrive/video_frames/VID3_11.jpg
writing /content/gdrive/MyDrive/video_frames/VID3_12.jpg
writing /content/gdrive/MyDrive/video_frames/VID3_13.jpg
writing /content/gdrive/MyDrive/video_frames/VID3_14.jpg
writing /content/gdrive/MyDrive/video_frames/VID3_15.jpg
writing /content/gdrive/MyDrive/video_frames/VID3_16.jpg
writing /content/gdrive/MyDrive/video_frames/VID3_17.jpg
writing /content/gdrive/MyDrive/video_fr

In [ ]:
# delete the folder storing the cropped images.
output_folder_path = "/content/gdrive/MyDrive/cropped_images"
if os.path.exists(output_folder_path):
    # Use shutil.rmtree to delete the directory
    shutil.rmtree(output_folder_path)
    print(f"Directory {output_folder_path} deleted successfully.")

Directory /content/gdrive/MyDrive/cropped_images deleted successfully.


For each frame in video_frames folder, crop the Bus stop sign board, by taking the coordinates detected by yolov7 model and store it into cropped_images folder.

In [ ]:
# Define the paths to the folders containing the images and text files
text_folder_path = results_path + "/labels"

# Load the image files
image_files = os.listdir(folder_path)

if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Load the text files and extract the class labels and coordinates
for text_file_name in os.listdir(text_folder_path):
    text_file_path = os.path.join(text_folder_path, text_file_name)
    with open(text_file_path, "r") as f:
        lines = f.readlines()
    for line in lines:
        # Extract the class label and coordinates
        class_label, x1, y1, x2, y2 = line.strip().split(" ")

        # Crop the regions of the image corresponding to the Bus stop sign board class labels
        if(int(class_label) == 1):
          for image_file_name in image_files:
              if image_file_name.startswith(text_file_name[:-4]):
                  image_file_path = os.path.join(folder_path, image_file_name)
                  print(image_file_path)
                  img = cv2.imread(image_file_path)
                  height, width, _ = img.shape
                  xmin = int((float(x1) - float(x2)/2) * width)
                  ymin = int((float(y1) - float(y2)/2) * height)
                  xmax = int((float(x1) + float(x2)/2) * width)
                  ymax = int((float(y1) + float(y2)/2) * height)
                  cropped_img = img[ymin:ymax, xmin:xmax]
                  output_file_name = "{}_{}_{}_{}_{}.jpg".format(image_file_name[:-4], class_label, x1, y1, x2, y2)
                  output_file_path = os.path.join(output_folder_path, output_file_name)
                  print(f"Writing file {output_file_path}")
                  cv2.imwrite(output_file_path, cropped_img)


/content/gdrive/MyDrive/video_frames/VID3_1.jpg
Writing file /content/gdrive/MyDrive/cropped_images/VID3_1_1_0.451852_0.423177_0.0296296.jpg
/content/gdrive/MyDrive/video_frames/VID3_10.jpg
Writing file /content/gdrive/MyDrive/cropped_images/VID3_10_1_0.451852_0.423177_0.0296296.jpg
/content/gdrive/MyDrive/video_frames/VID3_11.jpg
Writing file /content/gdrive/MyDrive/cropped_images/VID3_11_1_0.451852_0.423177_0.0296296.jpg
/content/gdrive/MyDrive/video_frames/VID3_12.jpg
Writing file /content/gdrive/MyDrive/cropped_images/VID3_12_1_0.451852_0.423177_0.0296296.jpg
/content/gdrive/MyDrive/video_frames/VID3_13.jpg
Writing file /content/gdrive/MyDrive/cropped_images/VID3_13_1_0.451852_0.423177_0.0296296.jpg
/content/gdrive/MyDrive/video_frames/VID3_14.jpg
Writing file /content/gdrive/MyDrive/cropped_images/VID3_14_1_0.451852_0.423177_0.0296296.jpg
/content/gdrive/MyDrive/video_frames/VID3_15.jpg
Writing file /content/gdrive/MyDrive/cropped_images/VID3_15_1_0.451852_0.423177_0.0296296.jpg
/

In [ ]:
# !sudo apt-get install tesseract-ocr

In [ ]:
# !pip install pytesseract

In [ ]:
# import cv2
# from google.colab.patches import cv2_imshow
# # Retrieve image dimensions
# image = cv2.imread("/content/gdrive/MyDrive/smalldata/file_400100.jpg")
# height, width, _ = image.shape

# # Normalize YOLO coordinates to [0, 1]
# x, y, w, h = 0.472687, 0.332230, 0.071349, 0.081678

# # Calculate bounding box coordinates
# xmin = int((x - w/2) * width)
# ymin = int((y - h/2) * height)
# xmax = int((x + w/2) * width)
# ymax = int((y + h/2) * height)

In [ ]:
# cimg = cv2.imread("/content/gdrive/MyDrive/cropped_images/VID3_488_1_0.516898_0.485938_0.10787.jpg")

In [ ]:
# cv2_imshow(cimg)

Preprocess the image before performing OCR.

In [ ]:
import numpy as np
from PIL import Image
def preprocessImage(img):
    # Resize Image
    img = cv2.resize(img, None, fx=1.2, fy=1.2, interpolation=cv2.INTER_CUBIC)
    # Change Color Format
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Kernel to filter image
    kernel = np.ones((1, 1), np.uint8)
    # Dilate + Erode image using kernel
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)
    img = cv2.addWeighted(img, 4, cv2.blur(img, (30, 30)), -4, 128)
    return img

In [ ]:
# cv2_imshow(preprocessImage(cimg))

In [ ]:
# cimg1 = preprocessImage(cimg)

In [ ]:
!pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import easyocr
# reader = easyocr.Reader(['en'])
# result = reader.readtext(cimg1)

# # Print results
# for detection in result:
#     text = detection[1]
#     print(text)

In [ ]:
# mapping from bus stop number to bus stop names
stops = {
    1: "CRI Deck",
    3: "Duke Centennial Hall",
    5: "EPIC - South",
    6: "Athletics Complex - East",
    8: "Student Union East",
    9: "Auxiliary Services",
    11: "Student Health - North",
    12: "Martin Hall",
    13: "Lot 6",
    14: "Lot 5A",
    15: "East Deck 2",
    16: "Fretwell - North",
    17: "Science Building",
    18: "Student Union West",
    20: "Athletics Complex - West",
    21: "EPIC - North",
    22: "Grigg Hall",
    23: "PORTAL",
    24: "Hunt Hall",
    30: "Reese East",
    31: "Robinson Hall North",
    32: "Cato Hall - North",
    33: "Student Union Deck",
    34: "Wallis Hall",
    35: "Student Health East",
    37: "Fretwell South",
    38: "Cato Hall - South",
    39: "Robinson Hall - South",
    40: "Levine Hall",
    41: "FM/PPS",
    42: "Light Rail",
    43: "Belk Hall",
    44: "Reese West",
    45: "Cone Deck",
    46: "South Village Deck",
    47: "Gage UA Center",
    48: "North Deck",
    49: "Alumni Way - East",
    55: "Motorsports",
    56: "Alumni Way West",
    57: "Football Stadium/Gate 1"
}


For each cropped Bus stop sign board, preprocess the image and pass it to easyocr to perform OCR and fetch the bus stop number from sign board. Bus stop number that is detected maximum number of times across all the images is the bus stop present in the input video.

In [ ]:
import easyocr

# Create an instance of the OCR reader
reader = easyocr.Reader(['en'])


# Get a list of all image files in the folder
image_files = [os.path.join(output_folder_path, f) for f in os.listdir(output_folder_path) if f.endswith('.jpg') or f.endswith('.jpeg') or f.endswith('.png')]

# Initialize a dictionary to store the counts of each number detected
number_counts = {}

# Loop through the image files
for image_file in image_files:
    # Perform OCR on the image
    result = reader.readtext(image_file)

    # Extract numbers from the OCR result and count their occurrences
    numbers = [int(re.sub("[^0-9]", "", detection[1])) for detection in result if re.match("^[0-9]+$", detection[1])]
    for number in numbers:
        print(f"{number} is detected")
        if number in number_counts:
            number_counts[number] += 1
        else:
            number_counts[number] = 1

# Find the number that was detected the maximum number of times
stopname = "Couldn't detect"
if number_counts:
    max_count_number = max(number_counts, key=number_counts.get)
    print(f"The number {max_count_number} was detected {number_counts[max_count_number]} times.")
    if(max_count_number > 5):
      try:
        stopname = stops[max_count_number]
        print(f"Bus stop {stopname} is approaching")
      except KeyError:
        print("Couldn't find bus stop")
    else:
      print("Couldn't find bus stop")
else:
    print("No numbers were detected in any of the images.")


8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
18 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
388 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
3105 is detected
8 is detected
8 is detected
5760 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
8 is detected
18 is detected
The number 8 was detected 62 times.
Bus stop Student